In [3]:
# pip install psychopy
# pip install psychopy --no-deps

In [35]:
from psychopy import visual
import pandas as pd
from PIL import Image
import time

In [2]:
features_path = 'Data/testfeature.xlsx'

In [3]:
file = pd.read_excel(features_path)
file.head()

,Image Path,Body,Arms,Legs,Eyes,Mouth,Antenna,Tail
0,Data/Images/brain.jpeg,2,3,4,NaN,NaN,NaN,NaN
1,data1,5,8,7,NaN,NaN,NaN,NaN
2,data2,6,6,8,NaN,NaN,NaN,NaN


In [4]:
img_path = file['Image Path'][0]

In [34]:
window = visual.Window([800, 600], monitor='testMonitor')

img = visual.ImageStim(window, image=img_path, pos=(0.5, 0))
circle = visual.Circle(win=window, radius=0.01,pos=(0.5, 0), color='red')

stimList = [img, circle]

screenshot = visual.BufferImageStim(window, stim=stimList)

screenshot.draw()
window.flip()

time.sleep(2)

window.close()

In [ ]:
def get_aliens(features_path):
    file = pd.read_excel(features_path)
    
    for index, row in file.iterrows():
        

In [ ]:
# def split_sprite_sheet(sprites_path):
#     sprite_sheet = Image.open(sprites_path)
#     count = 0
    
#     for y in range():
#         for x in range():